In [8]:
import pandas as pd 
import requests
import json
import os
from dotenv import load_dotenv
import ast
import gspread
import re
import time
from datetime import datetime

In [9]:
#загружаем env
load_dotenv()
google_credentials = os.getenv('GOOGLE_CREDENTIALS')
credentials = ast.literal_eval(google_credentials)
gc = gspread.service_account_from_dict(credentials)
sh = gc.open('flowwow')

In [10]:
worksheets_name = ['sem_tort', 'sem_b_tort', 'sem_hp_tort']
search_list = ['торт', 'бенто торт', 'торт на день рождение']

In [11]:
def clean_delivery_time(text):
    if pd.isna(text):
        return text
    
    # Replace &nbsp; with space
    text = re.sub(r'&nbsp;', ' ', text)
    
    # Remove ≈ if present
    text = re.sub(r'≈\s*', '', text)
    
    # Remove <br> tags and replace with space
    text = re.sub(r'<br>\s*', ' ', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Special case for time ranges (19:15 - 19:45)
    text = re.sub(r'(\d{1,2}:\d{2})\s*-\s*(\d{1,2}:\d{2})', r'\1-\2', text)
    
    return text

In [ ]:
def get_data(search_name):
    # Base URL
    url = "https://clientweb.flowwow.com/apiuser/products/search/"

    # Headers
    headers = {
        "authority": "clientweb.flowwow.com",
        "accept": "application/json",
        "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
        "cache-control": "no-cache",
        "canonical-url": "https://clientweb.flowwow.com",
        "referer": "https://clientweb.flowwow.com/",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    # Parameters
    params = {
        "property": json.dumps({
            "range_group_ids": [],
            "query": search_name,
            "city": 1,
            "lat": 55.783514,
            "lng": 37.720232
        }),
        "limit": 60,
        "filters": "{}",
        "currency": "RUB",
        "lang": "ru",
        "page": 1,
       # "guestId": "978682871072267100"
    }

    try:
        # Make the GET request
        response = requests.get(url, params=params, headers=headers)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Print the JSON response
            pass
        else:
            print(f"Request failed with status code {response.status_code}")
            print(response.text)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

    j = response.json()
    df = pd.json_normalize(j['data']['items'])
    new_df = pd.DataFrame(df)
    new_df['deliveryTime'] = new_df['deliveryTime'].apply(clean_delivery_time)
    new_df = new_df.drop(columns=['scoreKm', 'scorePoints'])
    new_df = new_df.fillna('')
    new_df['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M')
    return new_df

In [13]:
def load_new_data(new_df, sheet_name):
    wks = sh.worksheet(sheet_name)
    wks.clear()
    wks.update([new_df.columns.values.tolist()] + new_df.values.tolist())

In [14]:
for sh_name, search_name in zip(worksheets_name,  search_list):
    search_df = get_data(search_name)
    load_new_data(search_df, sh_name)
    time.sleep(1)

In [ ]:

pages = int(j['data']['total'] / 60)
for i in range(2, pages+1):
    print(i)

In [23]:
j['data']['total']

2000

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


NEW
